## ZE 기준 각 플라이트별 경쟁사의 최저 가격 비교
- 기준 사이트 : 인터파크
    * 대상 : ZE, 7C, TW, LJ
- 기간 : 내일 부터 21일간
- 오전 출근 후 생성 및 발송

In [1]:
import json
import logging
import datetime
import pandas as pd
from common.batch_util import crawl_period
from common.util import num_format, time_format, unique_list, json_loads, list_formatter
from common.log_util import log, logger_initialize
from common.parsing_util import parsing_json_data_to_dict
from common.web_util import make_grid_td, make_grid_th
from common.crawl_func import crawling_IP_dom
from common.mail_service_util import email_sender,email_multi_sender,make_newletter_html

logger_initialize('daily_report_logger_setting.json')

In [2]:
## 인터파크 정보 읽어오기
## 포맷 : 항공사,날짜,출발지,도착지,편명,출발시간,판매최저값
def scraping_IP_schdedule(raw_data):
    log('start scraping Interpark crawled data')
    raw_json = json_loads(raw_data)#json.loads(raw_data)
    if raw_json is None: ## 파싱 에러
        return None
    if raw_json['replyHeader']['errorCode'] == '1': ## 에러
        log('crawling data not found!',logging.ERROR)
        return None
    loop_field = "['replyAvailFare']['availFareSet']"
    parse_info = {
        'airline':"['replyAvailFare']['availFareSet'][{}]['segFare']['carCode']",#i
        'date':"['replyAvailFare']['availFareSet'][{}]['segFare']['depDate']",#i
        'flt':"['replyAvailFare']['availFareSet'][{}]['segFare']['mainFlt']",#i
        'dpt':"['replyAvailFare']['availFareSet'][{}]['segFare']['depCity']",#i
        'dpt_time':"['replyAvailFare']['availFareSet'][{}]['segFare']['depTime']",#i
        'arr':"['replyAvailFare']['availFareSet'][{}]['segFare']['arrCity']",#i
        'arr_time':"['replyAvailFare']['availFareSet'][{}]['segFare']['arrTime']",#i
        'fare1':"['replyAvailFare']['availFareSet'][{}]['segFare']['classDetail'][0]['fare']",#i,j
        'fare2':"['replyAvailFare']['availFareSet'][{}]['segFare']['classDetail'][1]['fare']",#i,j+1
        'tax':"['replyAvailFare']['availFareSet'][{}]['segFare']['airTax']",#i
        'fuel':"['replyAvailFare']['availFareSet'][{}]['segFare']['fuelChg']",#i
        'seat1':"['replyAvailFare']['availFareSet'][{}]['segFare']['classDetail'][0]['noOfAvailSeat']",#i,j
        'seat2':"['replyAvailFare']['availFareSet'][{}]['segFare']['classDetail'][1]['noOfAvailSeat']",#i,j+1
    }

    parsed_list = parsing_json_data_to_dict(raw_json,loop_field, parse_info)
    if len(parsed_list) == 0:
        log('scraping data not found!',logging.WARNING)
        return None
    
    scraped_list = [[d['airline'],d['date'],d['dpt'],d['arr'],d['flt'],d['dpt_time'],d['fare1']] for d in parsed_list ] +\
                    [[d['airline'],d['date'],d['dpt'],d['arr'],d['flt'],d['dpt_time'],d['fare2']] for d in parsed_list ]

    ## fare 숫자 변환
    for td in scraped_list:
        fare = td[-1]
        if fare is None or fare.strip() == '':
            fare = 0
        else:
            fare = int(fare.strip())
        td[-1] = fare

    log('end scraping Interpark crawled data')
    return scraped_list

## 값이 전부 0 이면 0
## 그렇지 않으면 0 이외의 값중 최소값 리턴
def get_min_value(f_list):
    sorted_list = sorted(f_list)
    if sorted_list[-1] <= 0:
        return 0
    return [v for v in sorted_list if v > 0][0]
## 조회해온 리스트에서 같은 편의 정보를 최소가격으로 조정
def unique_sch_fare_list(flist):
    u_sch_list = unique_list([td[:-1] for td in flist])
    fare_dict = {}
    for td in flist:
        ## flt를 키로 생성
        flt = fare_dict.get(td[4])
        ## flt에 대한 가격 리스트 처리
        if flt is None:
            fare_dict[td[4]] = [td[6]]
        else:
            flt.append(td[6])
    for td in u_sch_list:
        td.append(get_min_value(fare_dict[td[4]]))
    return u_sch_list
## 특정 시간에 대한 HHMM(시간분) 구조의 배열에서 시간 간격이 최소인 배열의 인덱스 찾기
## 인덱스와 차이값 리턴
def select_row_by_minute(hhmm,hhmm_list, reverse=False):
    ## 찾을 시간, 시간 배열, 최소/최대 여부 - 기본은 최소
    #print(hhmm)
    compare_list = []
    for i,l in enumerate(hhmm_list):
        compare_list.append([cal_minute(hhmm,l),i])
    return sorted(compare_list,reverse=reverse)[0][::-1]
## 시간 차이 분으로 계산 포맷 HHMM
def cal_minute(h1,h2):
    return abs((int(h1[:2])-int(h2[:2]))*60+int(h1[2:])-int(h2[2:]))
## 일일 리포트 생성
def make_dom_fare_daily_list(dpt,arr,dpt_date,airlines):
    ## 인터파크 이용 ZE 가격 정보 읽어오기
    raw_list = scraping_IP_schdedule(crawling_IP_dom('ZE',dpt,arr,dpt_date))
    if raw_list is None:
        return None
    ze_schedule_list = unique_sch_fare_list(raw_list)
    ## 인터파크 이용 각 항공사 가격 정보 리스트 생성
    ip_schedule_datas = []
    for airline in airlines:
        raw_list = scraping_IP_schdedule(crawling_IP_dom(airline,dpt,arr,dpt_date))
        ## 해당 항공사에 대한 크롤정보가 없을 경우 처리?
        if raw_list is None:
            continue;
        ip_schedule_datas.append(unique_sch_fare_list(raw_list))
    target_list = ze_schedule_list.copy()
    target_list = [tl[1:] for tl in target_list]
    ## ZE 스케줄을 기준으로 경쟁 항공사의 편, 가격 매칭
    for ip_schedule_list in ip_schedule_datas:
        for p,sch in enumerate(target_list):
            i,g = select_row_by_minute(sch[4],[sch[5] for sch in ip_schedule_list])
            target_list[p] = sch + [ip_schedule_list[i][0]+'-'+ip_schedule_list[i][4]+'-'+ip_schedule_list[i][5],ip_schedule_list[i][6]]
    return target_list
## 템플릿 이용 HTML 파일 작성
def make_daily_report_html(column_list,data_list):
    ## 검색조건 만들기
    title = 'daily report'
    column_html = make_grid_th(column_list)
    
    formats = [[0,lambda d:d[:4]+'-'+d[4:6]+'-'+d[6:]],[4,time_format],[5,num_format]]
    ## 경쟁사 리스트에 따른 포맷 처리를 위해 컬럼수를 이용하여 처리, 6번째 인덱스의 컬럼부터가 추가된 부분
    formats.extend([[7+2*i,num_format] for i in range(len(column_list[7::2]))])
    ## data_list가 column_list 보다 적을 경우 data_list의 사이즈 조정 빈값으로 채우기
    data_list = [td+[''for _ in range(len(column_list)-len(td))] for td in data_list]
    grid_html = make_grid_td(list_formatter(data_list,formats))
    with open('template/daily_fare_report.html',encoding='utf-8') as fp:
        template = fp.read()
    ## html 생성
    html = template.replace('{title}',title).replace('{columns}',column_html).replace('{grid_datas}',grid_html).replace('\n','')
    ## 파일 저장
    html_file = 'html/daily_fare_report_{}.html'.format(datetime.datetime.today().strftime('%Y%m%d%H%M%S%f'))
    # print(html_file)
    with open(html_file,'wt',encoding='utf-8') as fp:
        fp.write(html)
    return html_file

## 발송처리
def daily_report_mail_service(receivers,mail_msg,file):
    log('start email service')
    template = make_newletter_html(mail_msg)
    subject = 'rms daily reporting service'
    email_multi_sender(receivers,subject,template,attachs=[file])

In [3]:
## 가격 비교 파일 생성
dpt,arr = 'GMP','CJU'
airlines = ['7C','TW','LJ']
daily_report_list = []
for date in crawl_period(21):
    daily_list = make_dom_fare_daily_list(dpt,arr,date,airlines)
    if daily_list is None:
        continue;
    daily_report_list.extend(daily_list)
columns = ['date','dpt','arr','flt','dpt_time','fare']
for acode in airlines:
    columns.append(acode+'_info')
    columns.append(acode+'_fare')
report_file = make_daily_report_html(columns,daily_report_list)

2017-06-27 10:21:43,365 root     DEBUG    Crawling Interpark domastic schedule site


Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
inf:0 , dep:GMP , tripDivi:0 , airlineCode:ZE , chd:0 , adt:1 , arr:CJU , format:json , depDate:20170628 , 


2017-06-27 10:21:44,751 root     DEBUG    start scraping Interpark crawled data
2017-06-27 10:21:44,778 root     DEBUG    end scraping Interpark crawled data
2017-06-27 10:21:44,782 root     DEBUG    Crawling Interpark domastic schedule site


End Simple crawling
Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
inf:0 , dep:GMP , tripDivi:0 , airlineCode:7C , chd:0 , adt:1 , arr:CJU , format:json , depDate:20170628 , 


2017-06-27 10:21:46,508 root     DEBUG    start scraping Interpark crawled data
2017-06-27 10:21:46,545 root     DEBUG    end scraping Interpark crawled data
2017-06-27 10:21:46,557 root     DEBUG    Crawling Interpark domastic schedule site


...

End Simple crawling
Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
inf:0 , dep:GMP , tripDivi:0 , airlineCode:TW , chd:0 , adt:1 , arr:CJU , format:json , depDate:20170718 , 


2017-06-27 10:24:17,567 root     DEBUG    start scraping Interpark crawled data
2017-06-27 10:24:17,579 root     DEBUG    end scraping Interpark crawled data
2017-06-27 10:24:17,581 root     DEBUG    Crawling Interpark domastic schedule site


End Simple crawling
Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
inf:0 , dep:GMP , tripDivi:0 , airlineCode:LJ , chd:0 , adt:1 , arr:CJU , format:json , depDate:20170718 , 


2017-06-27 10:24:17,836 root     DEBUG    start scraping Interpark crawled data
2017-06-27 10:24:17,848 root     DEBUG    end scraping Interpark crawled data


End Simple crawling


In [6]:
## 파일 생성 오류시 재실행
report_file = make_daily_report_html(columns,daily_report_list)

In [5]:
## 생성된 HTML 파일 메일 발송 처리
#report_file = 'html/daily_fare_report_20170621080153395703.html'
mail_msg = {
    'title1' : 'RMS 뉴스 레터 - 온라인 경쟁사 가격 비교[기능,노선 추가]',
    'title2' : 'GMP-CJU(인터파크,항공사 21일 온라인 가격 비교',
    'date' : datetime.datetime.today().strftime("%Y-%m-%d"),
    'auth' : 'park363@eastarjet.com',
    'contents' : '''
    <p>온라인 상의 경쟁 항공사 가격 비교 입니다.</p>
    <p>작성일을 기준으로 다음날 부터 21일 동안의 가격을 읽어와 ZE 편 기준으로 시간차가</p>
    <p>가장 적은 경쟁사의 편에 대한 최저가를 표시 하였습니다.</p>
    <p>아침 출근후 받아 볼수 있도록 매일 아침 메일 서비스를 진행 합니다.</p>
    <br><br><p style="font-weight:bold;">[기능추가] 차트 기능 추가</p>
    <p>플라이트별 가격 비교 차트 기능을 추가하였습니다.</p>
    <p>상단부분에서 플라이트 선택후 차트 버튼을 클릭하면 해당 차트를 표출합니다.</p>
    <br><br><p style="font-weight:bold;">[노선추가] ICN-NRT 노선 추가중</p>
    <p>NRT 노선 추가 개발중에 있습니다.</p>
    <p>마무리후 다음주중에 추가하여 발송하도록 하겠습니다.</p>
    <br><br><p style="font-weight:bold;">[기타] 요청 사항</p>
    <p>노선을 추가하거나 기타 요청 사항이 있을 경우 김희정 과장과 상의 하시기 바랍니다.</p>
    <p>요청 사항은 적극 반영 하도록 하겠습니다.</p>
    <p>오전에 세미나 참여로 조금 늦었습니다.</p>
    '''
}
target = [
    '박종욱<park363@eastarjet.com>','우성주<topatsee@eastarjet.com>','조용현<ycho1211@eastarjet.com>','염희석<hsyoum@eastarjet.com>',
    '김희정<amykim@eastarjet.com>','정희진<tokkaipi@eastarjet.com>','안병준<bjahn@eastarjet.com>','장혜란<jhyeran@eastarjet.com>',
    '이동현<ldh1@eastarjet.com>','이재은<leeje@eastarjet.com>','임서윤<lhj01@eastarjet.com>','황혜진<hhj@eastarjet.com>',
    '안경미<km9980@eastarjet.com>','권재혁<jhkwon@eastarjet.com>'
]
test = ['박종욱<park363@eastarjet.com>']
receivers = target
daily_report_mail_service(receivers,mail_msg,report_file)

2017-06-27 10:55:53,282 root     DEBUG    start email service
2017-06-27 10:55:53,287 root     DEBUG    start multi email sender
2017-06-27 10:55:53,290 root     DEBUG    receiver :: 박종욱<park363@eastarjet.com> ::  ## email - :: rms daily reporting service
2017-06-27 10:55:54,031 root     DEBUG    receiver :: 우성주<topatsee@eastarjet.com> ::  ## email - :: rms daily reporting service
2017-06-27 10:55:54,713 root     DEBUG    receiver :: 조용현<ycho1211@eastarjet.com> ::  ## email - :: rms daily reporting service
2017-06-27 10:55:55,482 root     DEBUG    receiver :: 염희석<hsyoum@eastarjet.com> ::  ## email - :: rms daily reporting service
2017-06-27 10:55:56,421 root     DEBUG    receiver :: 김희정<amykim@eastarjet.com> ::  ## email - :: rms daily reporting service
2017-06-27 10:55:57,041 root     DEBUG    receiver :: 정희진<tokkaipi@eastarjet.com> ::  ## email - :: rms daily reporting service
2017-06-27 10:55:57,665 root     DEBUG    receiver :: 안병준<bjahn@eastarjet.com> ::  ## email - :: rms daily r

In [4]:
## 일일 데이터 생성
dpt,arr,dpt_date = 'GMP','CJU','20170622'
airlines = ['7C','TW','LJ']
result_list = make_dom_fare_daily_list(dpt,arr,dpt_date,airlines)

2017-06-22 10:37:06,129 root     DEBUG    Crawling Interpark domastic schedule site


Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
chd:0 , tripDivi:0 , airlineCode:ZE , adt:1 , depDate:20170622 , inf:0 , dep:GMP , format:json , arr:CJU , 


2017-06-22 10:37:28,423 root     DEBUG    start scraping Interpark crawled data
2017-06-22 10:37:28,435 root     DEBUG    end scraping Interpark crawled data
2017-06-22 10:37:28,437 root     DEBUG    Crawling Interpark domastic schedule site


End Simple crawling
Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
chd:0 , tripDivi:0 , airlineCode:7C , adt:1 , depDate:20170622 , inf:0 , dep:GMP , format:json , arr:CJU , 


2017-06-22 10:37:30,194 root     DEBUG    start scraping Interpark crawled data
2017-06-22 10:37:30,203 root     DEBUG    end scraping Interpark crawled data
2017-06-22 10:37:30,205 root     DEBUG    Crawling Interpark domastic schedule site


End Simple crawling
Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
chd:0 , tripDivi:0 , airlineCode:TW , adt:1 , depDate:20170622 , inf:0 , dep:GMP , format:json , arr:CJU , 


2017-06-22 10:37:30,644 root     DEBUG    start scraping Interpark crawled data
2017-06-22 10:37:30,651 root     DEBUG    end scraping Interpark crawled data
2017-06-22 10:37:30,653 root     DEBUG    Crawling Interpark domastic schedule site
2017-06-22 10:37:30,791 root     DEBUG    start scraping Interpark crawled data
2017-06-22 10:37:30,796 root     DEBUG    end scraping Interpark crawled data


End Simple crawling
Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
chd:0 , tripDivi:0 , airlineCode:LJ , adt:1 , depDate:20170622 , inf:0 , dep:GMP , format:json , arr:CJU , 
End Simple crawling


In [20]:
dpt,arr,dpt_date = 'GMP','CJU','20170622'
airline = '7C'
crawling_IP_dom(airline,dpt,arr,dpt_date)

2017-06-20 10:17:48,395 root     DEBUG    Crawling Interpark domastic schedule site


Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
format:json , dep:GMP , tripDivi:0 , depDate:20170622 , adt:1 , inf:0 , arr:CJU , chd:0 , airlineCode:7C , 
http://domair.interpark.com/api/booking/airJourney.do?format=json&dep=GMP&tripDivi=0&depDate=20170622&adt=1&inf=0&arr=CJU&chd=0&airlineCode=7C
End Simple crawling


'{"replyHeader":{"errorCode":"0","errorDesc":"Successful","agtCode":"interpark0","tripType":"OW","dep1":"GMP","arr1":"CJU","depDate1":"20170622","adt":"1","chd":"0","inf":"0","tripDivi":"0","fareKind":"A"},"noOfReplyAvaiFare":"1","replyAvailFare":{"depCity":"GMP","depDesc":"김포","arrCity":"CJU","arrDesc":"제주","noOfAvail":"20","availFareSet":[{"noOfSeg":"1","segFare":{"carCode":"7C","carDesc":"제주항공","carEngDesc":"Jeju Airline","opCarCode":"","opCarDesc":"","mainFlt":"151","opFlt":"","depCity":"GMP","depDesc":"김포","arrCity":"CJU","arrDesc":"제주","depDate":"20170622","depTime":"0625","depDay":"THU","arrDate":"20170622","arrTime":"0730","arrDay":"THU","currency":"KRW","fareType":"","fuelChg":"2200","chdFuelChg":"2200","airTax":"4000","chdAirTax":"2000","noOfClass":"2","classDetail":[{"fareRecKey":"adt%3D1%26chd%3D0%26inf%3D0%26DepartureDate0%3D20170622%26ArrivalDate0%3D20170622%26MktCar0%3D7C%26FlightNum0%3D151%26OriginLocation0%3DGMP%26DestinationLocation0%3DCJU%26BookingCls0%3DK%26ArrivalT

In [ ]:
columns = ['date','dpt','arr','flt','dpt_time','fare']
for acode in airlines:
    columns.append(acode+'_info')
    columns.append(acode+'_fare')
pd.DataFrame(result_list,columns=columns)